<a href="https://colab.research.google.com/github/mchablani/generative_ai/blob/main/colab_notebooks/stable_diffusion/layers_nft_recommendation/update_nft_hash_index.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:

from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
%ls /content/drive/MyDrive/expa/


askLangChainTestProject/  layers_data/              svb_gpt/
generative_ai_research/   mini_mini_imagenet_data/


In [3]:
%mkdir /content/drive/MyDrive/expa/layers_data


mkdir: cannot create directory ‘/content/drive/MyDrive/expa/layers_data’: File exists


In [4]:
%mkdir /content/drive/MyDrive/expa/layers_data/db

mkdir: cannot create directory ‘/content/drive/MyDrive/expa/layers_data/db’: File exists


In [5]:
%ls /content/drive/MyDrive/expa/layers_data


db/  images/  layer-nfts-sample-data.csv  nfts_dump.csv


In [6]:
import csv
import os
import requests
from pathlib import Path

In [7]:
db_folder_path = '/content/drive/MyDrive/expa/layers_data/db'
db_name = 'nft_hash.index'
db_meta_data_name = 'nft_hash_index_ids.pkl'

In [8]:
db_path = os.path.join(db_folder_path, db_name)
db_meta_data_path = os.path.join(db_folder_path, db_meta_data_name)


In [9]:
!pip install faiss-cpu

In [10]:
!pip install -q --upgrade transformers diffusers ftfy

In [11]:
! mkdir "/root/.huggingface"

mkdir: cannot create directory ‘/root/.huggingface’: File exists


In [12]:
! cp "drive/MyDrive/.huggingface/token" /root/.huggingface/token


In [13]:

from base64 import b64encode

import numpy
import os
import torch

# For video display:
from IPython.display import HTML
from matplotlib import pyplot as plt
from pathlib import Path
from PIL import Image
from torch import autocast
from torchvision import transforms as tfms
from tqdm.auto import tqdm
import torch.nn.functional as F
from transformers import logging
# import urllib.request
import requests
from io import BytesIO

torch.manual_seed(1)
if not (Path.home()/'.huggingface'/'token').exists(): notebook_login()

# Supress some unnecessary warnings when loading the CLIPTextModel
logging.set_verbosity_error()

# Set device
torch_device = "cuda" if torch.cuda.is_available() else "cpu"

In [14]:
torch.cuda.is_available()


True

In [15]:

# Load the pre-trained ViT model
model = torch.hub.load('facebookresearch/dino:main', 'dino_vits16')

# Define preprocessing pipeline for images
transform = tfms.Compose([
    tfms.Resize(256),
    tfms.CenterCrop(224),
    tfms.ToTensor(),
    tfms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])

Using cache found in /root/.cache/torch/hub/facebookresearch_dino_main


In [16]:
MODEL_EMBED_LEN = 384

In [17]:
def encode_image(img):
  with torch.no_grad():
    img = img.convert("RGB")
    img = transform(img).unsqueeze(0)
    encoded = model(img).squeeze().reshape(1, -1)
  return encoded

In [18]:
# check if db exists, if not create one

In [19]:
# for now always create one

In [20]:
# generate index
import numpy as np
import faiss
import pickle

In [21]:
index = faiss.IndexFlatL2(MODEL_EMBED_LEN)

In [22]:
id_index = faiss.IndexIDMap(index)
index_ids = {}


In [23]:
csv_folder_path = '/content/drive/MyDrive/expa/layers_data'
csv_name = 'nfts_dump.csv'

In [24]:
csv_path = os.path.join(csv_folder_path, csv_name)

In [32]:
with open(csv_path, newline='') as csvfile:
  reader = csv.DictReader(csvfile)
  row_index = 0
  for row in reader:
    row_index += 1
    url = row["media"]
    uid = row["uid"]
    try:
      # Send a request to download the image
      response = requests.get(url)
      response.raise_for_status()  # Check for any request errors

      # Extract the filename from the URL
      if len(response.content) > 100:
        image = Image.open(BytesIO(response.content))
        if image.format in ("PNG","JPEG","JPG"):
          encoded = encode_image(image)
          index_ids[row_index] = str(uid)
          id_index.add_with_ids(encoded.cpu(), np.array(row_index))
        else:
          print(f"Filter out:  image.format: {image.format}, url: {url}, row_index: {row_index}, uid: {uid}")
      else:
          print(f"Filter out:  url: {url}, row_index: {row_index}, uid: {uid}")
    except Exception as e:
      print(f"Error processing url: {url}, row_index: {row_index}, uid: {uid}")
      # break
      continue



Filter out:  image.format: GIF, url: https://res.cloudinary.com/layernft/image/fetch/f_gif,fl_lossy/q_50/https://ipfs.io/ipfs/QmYQVMu2JccRVZ7HCS9znjLMAVm2X88BnhV5UGSnNA9mPA, row_index: 7, uid: 176945b5-644c-465d-a5a4-70646a83529f
Filter out:  image.format: GIF, url: https://res.cloudinary.com/layernft/image/fetch/f_gif,fl_lossy/q_50/https://ipfs.io/ipfs/QmWTHc5WJHckKEMRC9dETwN8yyHUzp8qvwDSdKYEdU7nfx, row_index: 10, uid: 1f54af7b-0095-4249-837f-47c9f07957e0
Filter out:  image.format: GIF, url: https://res.cloudinary.com/layernft/image/fetch/f_gif,fl_lossy/q_50/https://ipfs.io/ipfs/Qmf2SAtbgLABWTh5yydpycVYjiV81NzzFK4BioYZQZ21aa/nft.gif, row_index: 11, uid: 2153b94e-3ed9-4e63-a531-733618a942b2
Filter out:  image.format: GIF, url: https://res.cloudinary.com/layernft/image/fetch/f_gif,fl_lossy/q_50/https://ipfs.io/ipfs/Qmcaon3vqDFJvAtUp1mtdKxuMSQu3HWzJe9rNDPegmAivA, row_index: 14, uid: 29b2b722-1411-4d70-bb24-783bbb0fb3f9
Filter out:  image.format: GIF, url: https://res.cloudinary.com/layer

In [29]:
len(index_ids)

70

In [30]:
faiss.write_index(id_index, db_path)

with open(db_meta_data_path, 'wb') as f:
  pickle.dump(index_ids, f)